In [11]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorchModel
from sagemaker.model import ModelPackage

In [3]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
BUCKET_NAME = "wildfires"
model_filename = "model_resnet18"
model_path = f"models/{model_filename}.tar.gz"

In [4]:
env = {
    'MMS_DEFAULT_RESPONSE_TIMEOUT': '1000000'  # Set the timeout to 500 seconds or higher
}

model = PyTorchModel(
    model_data=f"s3://{BUCKET_NAME}/{model_path}",
    role=role,
    framework_version='2.0.0',
    py_version='py310',
    entry_point='inference.py',
    env=env
)

In [6]:
# Model Package
model_package = model.register(
    content_types=['application/json'],    # Input type JSON
    response_types=['application/json'],   # Output type JSON
    inference_instances=['ml.g5.xlarge'],  # Inference instance type
    transform_instances=['ml.g5.xlarge'],  # Transform instance type
    model_package_group_name='wildfires-model-group'  # Model Group Name
)

print('Model package ARN:', model_package.model_package_arn)


Model package ARN: arn:aws:sagemaker:eu-central-1:567821811420:model-package/wildfires-model-group/1


In [ ]:
# Approve Model Package
model_package.update_approval_status(
    approval_status='Approved',
    approval_description='Model Approved.'
)

In [13]:
# After Manual Approval check model's status
sagemaker_client = boto3.client('sagemaker')

model_package_arn = model_package.model_package_arn

response = sagemaker_client.describe_model_package(
    ModelPackageName=model_package_arn
)

response['ModelApprovalStatus']

'PendingManualApproval'

In [ ]:
# Deploy Model
predictor = model_package.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.xlarge'
)

**Check model with prediction_w_deployed_model.ipynb**